In [ ]:
import json
import csv
import os
import matplotlib.pyplot as plt

In [ ]:
root = "/home/arthurc/labels_1202/"
flaw_json_list = ["benchpress_flaw", "deadlift_flaw", "squat_flaw"]
flaw_dict = {}

with open(root + "squat_flaw" + ".json") as f:
    flaw_json = json.load(f)

s = "squat_flaw"
flaw_dict[s] = []
for val in flaw_json["data"]:
    lt = val["gt_labels"]["lift_type"]
    great_one_flag = False
    for k, v in lt.items():
        if "great one" in v:
            great_one_flag = True

    if great_one_flag and val["metadata"]["filename"] == "35585614_268385877252000_4562125974187016192_n":
        temp_dict = {}
        temp_dict["filename"] = val["metadata"]["filename"]
        temp_dict["folder"] = val["metadata"]["folder"]
        temp_dict["macro_label"] = val["metadata"]["macro_label"]
        flaw_dict[s].append(temp_dict)
        
print(json.dumps(flaw_dict, sort_keys=True, indent=4))

save_file_path = root + "great_ones.json"

os.makedirs(os.path.dirname(save_file_path), exist_ok=True)
with open(save_file_path, 'w') as f:
    json.dump(flaw_dict, f, indent=4, separators=(',', ': '), sort_keys=True)

In [ ]:
openpose_path = "/data0/Results/lifting_openpose/"
openpose_folder_list = ["squat", "benchpress", "deadlift"]

squat_rep_json_path = "/home/dennyh/squat_reps.json"

lt = "squat"

openpose_json_path = openpose_path + lt + "/"
file_list = next(os.walk(openpose_json_path))[1]

flaw_classify_dict = {}
flaw_classify_dict_pretty = {}

with open(squat_rep_json_path) as f:
    squat_rep_json = json.load(f)

video_name = "35585614_268385877252000_4562125974187016192_n"

rep_seg = squat_rep_json[video_name]
flaw_classify_dict[video_name] = []
flaw_classify_dict_pretty[video_name] = []
for d in rep_seg:
    frame_list = [d["start"], d["mid"]]
    start_bottom_frame = []

    left_side_flag = True
    no_depth_flag = False

    temp_dict = {}

    for i, frame in enumerate(frame_list, start = 0):
        openpose_frame_path = openpose_json_path + video_name + "/" + video_name + "_" + format(frame, '012d') + "_keypoints.json"

        with open(openpose_frame_path) as f:
            openpose_json = json.load(f)

        kp_2d = openpose_json["people"][0]["pose_keypoints_2d"]

        useful_kp = {}
        useful_kp["LHip"] = kp_2d[(12) * 3 : (12 + 1) * 3]
        useful_kp["RHip"] = kp_2d[(9) * 3 : (9 + 1) * 3]
        useful_kp["LKnee"] = kp_2d[(13) * 3 : (13 + 1) * 3]
        useful_kp["RKnee"] = kp_2d[(10) * 3 : (10 + 1) * 3]

        L_conf_sum = useful_kp["LHip"][2] + useful_kp["LKnee"][2]
        R_conf_sum = useful_kp["RHip"][2] + useful_kp["RKnee"][2]

        if L_conf_sum >= R_conf_sum and i == 0:
            left_side_flag = True
        elif L_conf_sum < R_conf_sum and i == 0:
            left_side_flag = False

        if left_side_flag == True:
            if useful_kp["LHip"][1] <= useful_kp["LKnee"][1] and i == 1:
                no_depth_flag = True
        else:
            if useful_kp["RHip"][1] <= useful_kp["RKnee"][1] and i == 1:
                no_depth_flag = True

    flaw_classify_dict[video_name].append(no_depth_flag)
    
    if no_depth_flag:
        flaw_classify_dict_pretty[video_name].append("No Depth")
    else:
        flaw_classify_dict_pretty[video_name].append("Flawless")

print(json.dumps(flaw_classify_dict_pretty, sort_keys=True, indent=4))

In [ ]:
squat_gt_labels_path = "/home/arthurc/labels_1202/squat_flaw.json"

with open(squat_gt_labels_path) as f:
    squat_gt_labels = json.load(f)

true_positive_count = 0
false_positive_count = 0
true_negative_count = 0
false_negative_count = 0

result_dict = {}
result_dict["TP"] = []
result_dict["TN"] = []
result_dict["FP"] = []
result_dict["FN"] = []

for fn, no_depth_list in flaw_classify_dict.items():
    for squat_gt in squat_gt_labels["data"]:
        if squat_gt["metadata"]["filename"] == fn and no_depth_list != []:
            
            temp_dict = {}
            temp_dict["filename"] = squat_gt["metadata"]["filename"]
            temp_dict["folder"] = squat_gt["metadata"]["folder"]
            temp_dict["macro_label"] = squat_gt["metadata"]["macro_label"]
            
            if "no depth" in squat_gt["gt_labels"]["lift_type"]["bottom"]:
                if no_depth_list[-1]:
                    true_positive_count += 1
                    result_dict["TP"].append(temp_dict)
        
                else:
                    false_negative_count += 1
                    result_dict["FN"].append(temp_dict)
                    
            else:
                if not no_depth_list[-1]:
                    true_negative_count += 1
                    result_dict["TN"].append(temp_dict)
        
                else:
                    false_positive_count += 1
                    result_dict["FP"].append(temp_dict)

print(json.dumps(result_dict, sort_keys=True, indent=4))